Note: all values in tables do not reflect actual patient data. 

In [1]:
from __future__ import division
from __future__ import unicode_literals
from __future__ import print_function
from nltk import lazyimport

import nltk, re, pprint
from nltk import word_tokenize, pos_tag, sent_tokenize
from nltk.corpus import wordnet as wn
import io

import numpy as np#for math processing ,need
from numpy import arange #to handle ranges 
import unicodedata
import random
from collections import defaultdict
import csv
import pandas
import xlrd

This first line will import the data, assuming the same columns the classifier had output. Make sure to change the name of the file to the correct one before proceeding. 

In [ ]:
dffirst = pandas.read_excel("FILE NAME") # change file name

In [3]:
dffirst

,NOTE_DEID,PAT_DEID,NOTE_CODE,General,Regional,Local
0,1,32,Anesthesia Procedure Notes,0,0,1
1,2,32,Operative Notes,0,1,1
2,4,32,Post-Op Note,0,0,1
3,6,45,Anesthesia Procedure Notes,1,0,0
4,7,45,Operative Notes,1,0,1
5,8,45,Operative Reports,1,0,1
6,9,20,Post-Op Note,0,0,1
7,1,20,Anesthesia Procedure Notes,1,0,0
8,1,20,Operative Notes,1,0,1
9,1,20,Operative Reports,1,0,1


,NOTE_DEID,PAT_DEID,NOTE_CODE,General,Regional,Local
0,1,32,Anesthesia Procedure Notes,0,0,1
1,2,32,Operative Notes,0,1,1
2,4,32,Post-Op Note,0,0,1
3,6,45,Anesthesia Procedure Notes,1,0,0
4,7,45,Operative Notes,1,0,1
5,8,45,Operative Reports,1,0,1
6,9,20,Post-Op Note,0,0,1
7,1,20,Anesthesia Procedure Notes,1,0,0
8,1,20,Operative Notes,1,0,1
9,1,20,Operative Reports,1,0,1


In [4]:
notetypes = dffirst.groupby("NOTE_CODE").count()
notetypes = notetypes.index
notetypes

Index([u'Anesthesia Procedure Notes', u'Operative Notes', u'Operative Reports',
       u'Post-Op Note'],
      dtype='object', name=u'NOTE_CODE')

Index([u'Anesthesia Procedure Notes', u'Operative Notes', u'Operative Reports',
       u'Post-Op Note'],
      dtype='object', name=u'NOTE_CODE')

In [5]:
df_bypatient = dffirst.pivot(index='PAT_DEID', columns='NOTE_CODE', values=['General', 'Regional', 'Local'])
df_bypatient.columns = df_bypatient.columns.swaplevel(0, 1)
df_bypatient.sort_index(level =0, axis=1,inplace=True)
df_bypatient

NOTE_CODE Anesthesia Procedure Notes                Operative Notes        \
                             General Local Regional         General Local   
PAT_DEID                                                                    
20                               1.0   0.0      0.0             1.0   1.0   
32                               0.0   1.0      0.0             0.0   1.0   
45                               1.0   0.0      0.0             1.0   1.0   

NOTE_CODE          Operative Reports                Post-Op Note        \
          Regional           General Local Regional      General Local   
PAT_DEID                                                                 
20             0.0               1.0   1.0      0.0          0.0   1.0   
32             1.0               NaN   NaN      NaN          0.0   1.0   
45             0.0               1.0   1.0      0.0          NaN   NaN   

NOTE_CODE           
          Regional  
PAT_DEID            
20             0.0  
32             0.0  
45             NaN

NOTE_CODE Anesthesia Procedure Notes                Operative Notes        \
                             General Local Regional         General Local   
PAT_DEID                                                                    
20                               1.0   0.0      0.0             1.0   1.0   
32                               0.0   1.0      0.0             0.0   1.0   
45                               1.0   0.0      0.0             1.0   1.0   

NOTE_CODE          Operative Reports                Post-Op Note        \
          Regional           General Local Regional      General Local   
PAT_DEID                                                                 
20             0.0               1.0   1.0      0.0          0.0   1.0   
32             1.0               NaN   NaN      NaN          0.0   1.0   
45             0.0               1.0   1.0      0.0          NaN   NaN   

NOTE_CODE           
          Regional  
PAT_DEID            
20             0.0  
32             0.0  
45             NaN

In [6]:
df_bypatient.index

Int64Index([20, 32, 45], dtype='int64', name=u'PAT_DEID')

Int64Index([20, 32, 45], dtype='int64', name=u'PAT_DEID')

Following cell for counting up #s of notes for each patient. 

In [7]:

df_bypatient["Total # of Notes for Patient"] = 0
df_bypatient["Patient ID"] = df_bypatient.index
df_bypatient = df_bypatient.reset_index()
for index in df_bypatient.index:
    n = len([val for val in pandas.isna(df_bypatient.iloc[index]) if val == True])/3
    total = len(notetypes) - n
    df_bypatient.loc[index,"Total # of Notes for Patient"] = total

The following cell starts a new dataframe that will take any data from the df_bypatient table that has at least a certain number of notes. 
Change 2 to a different number depending on how many notes you believe would be enough to show some consistency for a patient. 
E.g. if only patients with 4 or more notes are wanted, the 2 would be changed to a 3 (one less than 4).

In [8]:
alltypes = pandas.DataFrame()
alltypes = df_bypatient.loc[df_bypatient['Total # of Notes for Patient'] > 2] # change standard to desired value of notes 
alltypes["General Agreement"] = np.nan
alltypes["Regional Agreement"] = np.nan
alltypes["Local Agreement"] = np.nan

In [9]:
alltypes

NOTE_CODE PAT_DEID Anesthesia Procedure Notes                Operative Notes  \
                                      General Local Regional         General   
0               20                        1.0   0.0      0.0             1.0   
1               32                        0.0   1.0      0.0             0.0   
2               45                        1.0   0.0      0.0             1.0   

NOTE_CODE                Operative Reports                Post-Op Note        \
          Local Regional           General Local Regional      General Local   
0           1.0      0.0               1.0   1.0      0.0          0.0   1.0   
1           1.0      1.0               NaN   NaN      NaN          0.0   1.0   
2           1.0      0.0               1.0   1.0      0.0          NaN   NaN   

NOTE_CODE          Total # of Notes for Patient Patient ID General Agreement  \
          Regional                                                             
0              0.0                          4.0         20               NaN   
1              0.0                          3.0         32               NaN   
2              NaN                          3.0         45               NaN   

NOTE_CODE Regional Agreement Local Agreement  
                                              
0                        NaN             NaN  
1                        NaN             NaN  
2                        NaN             NaN

NOTE_CODE PAT_DEID Anesthesia Procedure Notes                Operative Notes  \
                                      General Local Regional         General   
0               20                        1.0   0.0      0.0             1.0   
1               32                        0.0   1.0      0.0             0.0   
2               45                        1.0   0.0      0.0             1.0   

NOTE_CODE                Operative Reports                Post-Op Note        \
          Local Regional           General Local Regional      General Local   
0           1.0      0.0               1.0   1.0      0.0          0.0   1.0   
1           1.0      1.0               NaN   NaN      NaN          0.0   1.0   
2           1.0      0.0               1.0   1.0      0.0          NaN   NaN   

NOTE_CODE          Total # of Notes for Patient Patient ID General Agreement  \
          Regional                                                             
0              0.0                          4.0         20               NaN   
1              0.0                          3.0         32               NaN   
2              NaN                          3.0         45               NaN   

NOTE_CODE Regional Agreement Local Agreement  
                                              
0                        NaN             NaN  
1                        NaN             NaN  
2                        NaN             NaN

In [10]:
#average the frequency of 0s and 1s among the different types of notes for each patient
for i, row in alltypes.iterrows():
    g = 0
    r = 0
    l = 0
    for notetype in notetypes:
        if pandas.isnull(row[notetype]["General"]) == False:
            g = g + row[notetype]["General"]
        if pandas.isnull(row[notetype]["Regional"]) == False:
            r = r + row[notetype]["Regional"]
        if pandas.isnull(row[notetype]["Local"]) == False:
            l = l + row[notetype]["Local"]     
    gg = g/row['Total # of Notes for Patient']
    rr = r/row['Total # of Notes for Patient']
    ll = l/row['Total # of Notes for Patient']
    gg = gg[0]
    rr = rr[0]
    ll = ll[0]


    alltypes.loc[i, "General Agreement"] = gg
    alltypes.loc[i, "Regional Agreement"] = rr
    alltypes.loc[i, "Local Agreement"] = ll

In [11]:
alltypes

NOTE_CODE PAT_DEID Anesthesia Procedure Notes                Operative Notes  \
                                      General Local Regional         General   
0               20                        1.0   0.0      0.0             1.0   
1               32                        0.0   1.0      0.0             0.0   
2               45                        1.0   0.0      0.0             1.0   

NOTE_CODE                Operative Reports                Post-Op Note        \
          Local Regional           General Local Regional      General Local   
0           1.0      0.0               1.0   1.0      0.0          0.0   1.0   
1           1.0      1.0               NaN   NaN      NaN          0.0   1.0   
2           1.0      0.0               1.0   1.0      0.0          NaN   NaN   

NOTE_CODE          Total # of Notes for Patient Patient ID General Agreement  \
          Regional                                                             
0              0.0                          4.0         20              0.75   
1              0.0                          3.0         32              0.00   
2              NaN                          3.0         45              1.00   

NOTE_CODE Regional Agreement Local Agreement  
                                              
0                   0.000000        0.750000  
1                   0.333333        1.000000  
2                   0.000000        0.666667

NOTE_CODE PAT_DEID Anesthesia Procedure Notes                Operative Notes  \
                                      General Local Regional         General   
0               20                        1.0   0.0      0.0             1.0   
1               32                        0.0   1.0      0.0             0.0   
2               45                        1.0   0.0      0.0             1.0   

NOTE_CODE                Operative Reports                Post-Op Note        \
          Local Regional           General Local Regional      General Local   
0           1.0      0.0               1.0   1.0      0.0          0.0   1.0   
1           1.0      1.0               NaN   NaN      NaN          0.0   1.0   
2           1.0      0.0               1.0   1.0      0.0          NaN   NaN   

NOTE_CODE          Total # of Notes for Patient Patient ID General Agreement  \
          Regional                                                             
0              0.0                          4.0         20              0.75   
1              0.0                          3.0         32              0.00   
2              NaN                          3.0         45              1.00   

NOTE_CODE Regional Agreement Local Agreement  
                                              
0                   0.000000        0.750000  
1                   0.333333        1.000000  
2                   0.000000        0.666667

In [12]:
import xlsxwriter
#change what dataframe to save, and what rows to save. for columns, can probably drop noteid but keep pat ID?
df = alltypes
writer = pandas.ExcelWriter('anes_results_by_note.xls',engine = "xlsxwriter")
df.to_excel(writer, sheet_name = 'Sheet1')
workbook = writer.book
worksheet = writer.sheets["Sheet1"]
format1 = workbook.add_format({'num_format': '1'})
worksheet.set_column('C:C', None, format1)
writer.save()